In [1]:
import openai
import re
import time
import numpy as np

from tqdm import tqdm
from pprint import pprint


from agents import SCAgent, initial_instructions, CounselorAgent, PatientAgent, ModeratorAgent, wprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [11]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#Note: This code sample requires OpenAI Python library version 0.28.1 or lower.

import openai
openai.api_type = "azure"
openai.api_base = "https://tfccgpoc.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

message_text = [{"role":"system","content":"you are a expert AI system"},{"role":"user","content":" what is openAI?"}]

completion = openai.ChatCompletion.create(
  engine="TFccgUsecase",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion['choices'][0]['message'])

{
  "role": "assistant",
  "content": "OpenAI is an artificial intelligence research laboratory consisting of the for-profit corporation OpenAI LP and its parent company, the non-profit OpenAI Inc. The company was founded in December 2015 by a group of technology luminaries including Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba. OpenAI's mission is to develop and promote friendly AI for the betterment of humanity, and to ensure that the development of artificial intelligence is aligned with human values and aligned with the long-term survival of the human race. OpenAI is known for its work in natural language processing, reinforcement learning, and robotics, among other areas."
}


***Counselor Init***

In [3]:
counselor_initial_dialog_history = initial_instructions('Genetic_counselor.txt')
counselor = CounselorAgent(initial_dialog_history=counselor_initial_dialog_history, agent_type="counselor", engine="gpt-3.5-turbo")
pprint(counselor.dialog_history)

[{'content': 'you are a expert genetic counsellor . please ask one question '
             'and wait for patient to answer before asking next question. '
             'cover all questions about family history, patient history and '
             'pathology report. \n'
             'after all questions are done explain how genetic testing works. '
             'your goal is to provide clear and relevant information about '
             'genetic risk factors in a supportive and educational '
             'environment. at the very end offer to schedule a appointment',
  'role': 'system'},
 {'content': 'The patient is a 71-year-old male who has smoked for the past 50 '
             'years',
  'role': 'user'}]


***Patient Init***

In [4]:
patient_initial_dialog_history = initial_instructions('Patient.txt')
patient = PatientAgent(initial_dialog_history=patient_initial_dialog_history, agent_type="patient", engine="gpt-3.5-turbo")
pprint(patient.dialog_history)

[{'content': 'You are a patient chatting with a doctor over an online chat '
             'interface. The counselor has never met you before. your '
             'background is -you are a 71-year-old male who has smoked for the '
             'past 50 years. you have  shortness of breath, coughing up blood '
             'and chest pain. A biopsy revealed that you have stage IIIB lung '
             'adenocarcinoma which measures about 4 cm in diameter located on '
             'his left upper lobe after undergoing wedge resection surgery. '
             'Pathology report showed an invasive acinar predominant subtype '
             'with secondary solid growth pattern and positive TTF-1 '
             'immunohistochemical stains but no specific mutations mentioned '
             '(EGFR, ALK, ROS1). your family history reveals both paternal '
             'grandfather and grandmother had a history of lung cancer.\n'
             'Only provide information that counselor asked for.',
  'r

***Moderator Init***

In [5]:
moderator_initial_dialog_history = initial_instructions("Moderator.txt")
moderator = ModeratorAgent(initial_dialog_history=moderator_initial_dialog_history, agent_type="moderator", engine="gpt-3.5-turbo")
pprint(moderator.dialog_history)

[{'content': 'Now enter the role-playing mode. In the following conversation, '
             'you will play as a moderator for a dialog.',
  'role': 'system'},
 {'content': 'In the following conversation, you will play as a moderator in a '
             'discussion between a genetic counselor and a patient. you have '
             'to evaluate if either of them are asking a question. if yes then '
             'the conversation is not completed. \n'
             'The conversation is complete if the counselor offers a '
             'appointment or if patient discusses a appointment\n'
             'conversation is also complete if counselor has offered details '
             'of genetic testing\n'
             'conversation is also complete if patient says thank you',
  'role': 'user'}]


In [8]:
def run(counselor, patient, moderator, n=6, fd=None, verbose=True):
    wprint('counselor: %s' % counselor.last_response, fd, verbose=verbose)
    wprint('patient: %s' % patient.last_response, fd, verbose=verbose)
    
    wprint('---- start conversation ----', fd, verbose=verbose)
    patient_run = patient.last_response
    start_involve_moderator = False
     
    moderate=''
    for _ in range(n):
        time.sleep(1)
        counselor_run = counselor.call(patient_run)
        wprint('counselor: %s \n' % counselor.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False ):
            start_involve_moderator = True
            #wprint('---- start moderating ----', fd, verbose=verbose)
        
        if(start_involve_moderator):
            time.sleep(1)
            moderate = moderator.moderate(counselor.dialog_history,who_was_last="counselor")
            #wprint('MODERATER has the conversation concluded? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                
                break
            else: pass

        time.sleep(1)   
        patient_run = patient.call(counselor_run)
        wprint('patient: %s \n' % patient.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False ):
            start_involve_moderator = True
            #wprint('---- start moderating ----', fd, verbose=verbose)
            
        if(start_involve_moderator):
            time.sleep(1)
            moderate = moderator.moderate( patient.dialog_history, who_was_last="patient")
            #wprint('MODERATER has the conversation concluded? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                
                break
            else: pass
                

    print("=======Moderator=====")
    print('\n %s' % moderate)
    if("yes" in moderate.lower()):
            print("conversation concluded")
            counselor_run = counselor.call("patient: what neogenomics panel is recommended for me?")
            wprint('counselor: %s' % counselor.last_response, fd, verbose=verbose)
    

In [9]:
patient.reset()
counselor.reset()
moderator.reset()
run(counselor, patient, moderator, n=6)
 

counselor: The patient is a 71-year-old male who has smoked for the past 50 years
patient: i am  a 71-year-old male who has smoked for the past 50 years
---- start conversation ----
counselor: Thank you for sharing that with me. Have you noticed any changes in your health recently? 

patient: Yes, I have been experiencing shortness of breath, coughing up blood and chest pain. 

counselor: I'm sorry to hear that. Have you seen a doctor about these symptoms? Have you had any medical tests or procedures done recently? 

patient: Yes, I have seen a doctor about these symptoms and undergone a biopsy. The biopsy revealed that I have stage IIIB lung adenocarcinoma which measures about 4 cm in diameter located on his left upper lobe after undergoing wedge resection surgery. Pathology report showed an invasive acinar predominant subtype with secondary solid growth pattern and positive TTF-1 immunohistochemical stains but no specific mutations mentioned (EGFR, ALK, ROS1). 

counselor: Thank you 